<a href="https://colab.research.google.com/github/rb58853/Music-Genre-Classificator/blob/main/Conv1D%20Model/Conv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [3]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 123074 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [1]:
import os

### Importaciones

In [4]:
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Activation
from keras.utils import to_categorical
import keras

### Lectura del dataset


In [5]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    return -1

def get_data(data_path, genres, decoder, training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    """
    data_path: se le pasa la direccion de la carpeta donde se encuentra la base de datos.
    genres: se le pasa una lista con los nombres da cada carpeta que contiene un genero dado.
    decoder: funcion para decodificar el dato que se le pasa, por ejemplo en caso de imagenes habria hacer imread
    """

    data_training = {'in': [], 'out': []}
    data_validation = {'in': [], 'out': []}
    data_test = {'in': [], 'out': []}

    for genre in genres:
        files = os.listdir(data_path + genre)
        count = len(files)

        for (filename, index) in zip(files, range(len(files))):
            filepath = data_path + genre + '/' + filename

            if (index < training_percentaje * count):
                data_training['in'].append(decoder(filepath))
                data_training['out'].append(index_genre(genre, genres))
                continue

            if (index < (training_percentaje + validation_percentaje) * count):
                data_validation['in'].append(decoder(filepath))
                data_validation['out'].append(index_genre(genre, genres))
            else:
                data_test['in'].append(decoder(filepath))
                data_test['out'].append(index_genre(genre, genres))

    data_training = {'in': np.array(data_training['in']),'out': np.array(data_training['out'])}
    data_validation = {'in': np.array(data_validation['in']),'out': np.array(data_validation['out'])}
    data_test = {'in': np.array(data_test['in']),'out': np.array(data_test['out'])}

    return {
        'data_training': data_training,
        'data_validation': data_validation,
        'data_testing': data_test
    }

In [6]:
data_path = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# CARGAR UN ARCHIVO DE AUDIO
def load_audio_file(file_path):
    input_length = 660000  # This is 15 seconds with 44100 sample rate
    data = librosa.core.load(file_path, sr=22050)[0]  # We use librosa to load audio file with sample rate 22050
    if len(data) > input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data

def get_data_conv1D(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1):
    return get_data(data_path, genres, load_audio_file,training_percentaje,validation_percentaje,test_percentaje)

data = get_data_conv1D()

In [7]:
X_train = data['data_training']['in']
y_train = data['data_training']['out']
X_val = data['data_validation']['in']
y_val = data['data_validation']['out']
X_test = data['data_testing']['in']
y_test = data['data_testing']['out']

### Creacion del modelo

In [8]:
# MODELO DE CNN 1D BASADO EN LA ARQUITECTURA ResNet
model = Sequential()
# PRIMERA CAPA DE CONVOLUCION
model.add(Conv1D(32, kernel_size=3, strides=1, padding="same", input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
# BLOQUES RESIDUALES
for _ in range(1):
    model.add(Conv1D(32, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(32, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
# CAPA DE POOLING
model.add(MaxPooling1D(pool_size=2, strides=2))
# CAPA CONECTADA COMPLETAMENTE
model.add(Flatten())
# model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))  # Assuming 10 genres
# COMPILACION DEL MODELO
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

### Train

In [10]:

# REMODELAR LOS DATOS
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(X_train, y_train, batch_size=80, epochs=100, validation_data=(X_val, y_val))
model_name = 'conv1D_model.h5'
model.save('/content/drive/MyDrive/Ensemble/'+ str(model_name))

ValueError: ignored

### Testing

In [ ]:
data

input = data['data_testing']['in']
output = data['data_testing']['out']

path = '/content/drive/MyDrive/Ensemble/' + model_name

model = keras.models.load_model(path)

score = model.evaluate(input, output, verbose=0)
# print('testing loss: ' + str(score[0]))
print('testing accuracy: ' + str(score[1]))

input = data['data_validation']['in']
output = data['data_validation']['out']
score = model.evaluate(input, output, verbose=0)
# print('validation loss: ' + str(score[0]))
print('validation accuracy: ' + str(score[1]))

input = data['data_training']['in']
output = data['data_training']['out']
score = model.evaluate(input, output, verbose=0)
# print('training loss: ' + str(score[0]))
print('training accuracy: ' + str(score[1]))


# Liberacion de recursos

In [ ]:
!pip install numba

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device = cuda.select_device(0)

device.reset()


In [ ]:
!service networking restart

networking: unrecognized service


In [ ]:
!kill -9 -1